In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split

from tqdm.keras import TqdmCallback
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import datetime

In [6]:
# Train
size = 'demo'
type_ = 'train'

numpy_file_X_path = f'./files/numpy/X_{type_}_{size}.npy'
numpy_file_y_path = f'./files/numpy/y_{type_}_{size}.npy'

X_train = np.load(numpy_file_X_path)
y_train = np.load(numpy_file_y_path)

In [9]:
# Validation 
size = 'demo'
type_ = 'validation'

numpy_file_X_path = f'./files/numpy/X_{type_}_{size}.npy'
numpy_file_y_path = f'./files/numpy/y_{type_}_{size}.npy'

X_val = np.load(numpy_file_X_path)
y_val = np.load(numpy_file_y_path)

In [10]:
# Use only the first 100 rows for testing
X_train = X_train[:80]
y_train = y_train[:80]
X_val = X_val[:20]
y_val = y_val[:20]

In [35]:
# Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# print(X_train.shape)
# print(X_test.shape)
# print(y_train.shape)
# print(y_test.shape)

(80, 600)
(20, 600)
(80,)
(20,)


In [11]:
# Define the model
model = Sequential([
    tf.keras.Input(shape=(600,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation = 'linear')
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])


In [12]:

# Prepare TensorBoard callback
log_dir = "files/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model
num_epochs = 2
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=16,
                    validation_data=(X_val, y_val),
                    callbacks=[tensorboard_callback, TqdmCallback(verbose=1)],
                    verbose=2)

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Epoch 1/2
5/5 - 0s - loss: 0.6904 - mse: 0.6904 - val_loss: 0.2678 - val_mse: 0.2678
Epoch 2/2
5/5 - 0s - loss: 0.1430 - mse: 0.1430 - val_loss: 0.2380 - val_mse: 0.2380


In [ ]:
# Evaluate the model
train_loss, train_mse = model.evaluate(X_train, y_train, verbose=1)
test_loss, test_mse = model.evaluate(X_val, y_val, verbose=1)

In [15]:
# Save the 
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_file_path = f'./files/models/model_{now}.h5py'
model.save(model_file_path)

INFO:tensorflow:Assets written to: ./files/models/model_20240617-195008.h5py\assets
